# 🧑‍⚕️ NeMo Data Designer: Realistic Patient Data & Physician Notes

> ⚠️ **Warning**: NeMo Data Designer is currently in Early Release and is not recommended for production use.

#### 📚 What you'll learn

This notebook demonstrates how to use NeMo Data Designer to generate realistic patient data including physician notes.\
 We'll leverage both structured data generation and LLM capabilities to create a comprehensive medical dataset.

The dataset includes:
- Policy and claim details
- Policyholder and claimant information (PII)
- Claim descriptions and adjuster notes with embedded PII
- Medical information for relevant claims


<br>

> 👋 **IMPORTANT** – Environment Setup
>
> - If you haven't already, follow the instructions in the [README](../../../README.md) to install the necessary dependencies.
>
> - You may need to restart your notebook's kernel after setting up the environment.
> - In this notebook, we assume you have a self-hosted instance of Data Designer up and running.
>
> - For deployment instructions, see the [Installation Options](https://docs.nvidia.com/nemo/microservices/latest/design-synthetic-data-from-scratch-or-seeds/index.html#installation-options) section of the [NeMo Data Designer documentation](https://docs.nvidia.com/nemo/microservices/latest/design-synthetic-data-from-scratch-or-seeds/index.html).


### 📦 Import the essentials

- The `data_designer` module of `nemo_microservices` exposes Data Designer's high-level SDK.

- The `essentials` module provides quick access to the most commonly used objects.


In [ ]:
from nemo_microservices.data_designer.essentials import (
    CategorySamplerParams,
    DataDesignerConfigBuilder,
    InferenceParameters,
    LLMTextColumnConfig,
    ModelConfig,
    NeMoDataDesignerClient,
    PersonSamplerParams,
    SamplerColumnConfig,
    SamplerType,
    SeedDatasetReference,
    UUIDSamplerParams,
)

### ⚙️ Initialize the NeMo Data Designer Client

- `NeMoDataDesignerClient` is responsible for submitting generation requests to the microservice.


In [ ]:
NEMO_MICROSERVICES_BASE_URL = "http://localhost:8080"

data_designer_client = NeMoDataDesignerClient(base_url=NEMO_MICROSERVICES_BASE_URL)

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see [the model config docs](https://docs.nvidia.com/nemo/microservices/latest/design-synthetic-data-from-scratch-or-seeds/configure-models.html) for more details).

- By default, the microservice uses [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [ ]:
# This name is set in the microservice deployment configuration.
MODEL_PROVIDER = "nvidiabuild"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nvidia-nemotron-nano-9b-v2"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v2"

# This sets reasoning to False for the nemotron-nano-v2 model.
SYSTEM_PROMPT = "/no_think"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.6,
            top_p=0.95,
            max_tokens=1024,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [ ]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🌱 Loading Seed Data

- We'll use the symptom-to-diagnosis dataset as our seed data. 

- This dataset contains patient symptoms and corresponding diagnoses which will help generate realistic medical scenarios.

<br> 

> 🌱 **Why use a seed dataset?**
>
> - Seed datasets let you steer the generation process by providing context that is specific to your use case.
>
> - Seed datasets are also an excellent way to inject real-world diversity into your synthetic data.
>
> - During generation, prompt templates can reference any of the seed dataset fields.

<br>

> 💡 **About datastores**
>
> - You can use seed datasets from _either_ the Hugging Face Hub or a locally deployed datastore.
>
> - By default, we use the local datastore deployed with the Data Designer microservice.
>
> - The datastore endpoint is specified in the deployment configuration.


👋 **Note**: At this time, we only support using a single file as the seed. If you have multiple files you would like to use as \
seeds, it is recommended you consolidated these into a single file. 

In [ ]:
from datasets import load_dataset

# Let's use the symptom-to-diagnosis dataset to seed our workflow
df_seed = load_dataset("gretelai/symptom_to_diagnosis")["train"].to_pandas()
df_seed = df_seed.rename(columns={"output_text": "diagnosis", "input_text": "patient_summary"})

print(f"Number of records: {len(df_seed)}")

df_seed.head()

In [ ]:
# Upload the dataset to the local datastore if not already present.
# NDD suppoorts uploading pandas DataFrame or a CSV, Parquet, or JSON file to the datastore
seed_dataset_reference = data_designer_client.upload_seed_dataset(
    dataset=df_seed,
    repo_id="data-designer/demo",
    datastore_settings={"endpoint": "http://localhost:3000/v1/hf"},
)

# Pass the reference to the config builder for use during generation.
config_builder.with_seed_dataset(seed_dataset_reference)

> 💡 **Tip**
>
> - If the dataset already exists in the datastore, you can create the seed dataset reference directly.
>
> - See the [seed dataset docs](https://docs.nvidia.com/nemo/microservices/latest/design-synthetic-data-from-scratch-or-seeds/seed-datasets.html) for more info.
>
> <br>
>
> For example:
>
> ```python
> from nemo_microservices.data_designer.essentials import SeedDatasetReference
>
> # Create reference to existing dataset in the datastore.
> seed_dataset_reference = SeedDatasetReference(
>     dataset="data-designer/demo/gretelai_symptom_to_diagnosis.csv",
>     datastore_settings={"endpoint": "http://localhost:3000/v1/hf"},
> )
> ```

## 🎲 Creating Person Samplers

- We create persona samplers to simulate details about the patient and the doctor 

- The persona samplers allow you to sample realistic details of individuals using a model trained on the US Census.\
If the locale of the persona you are generating is anything other than `en_US`, then the personas will be generated using Faker


In [ ]:
# Create a couple random person samplers.
config_builder.add_column(
    SamplerColumnConfig(
        name="patient_sampler",
        sampler_type=SamplerType.PERSON,
        params=PersonSamplerParams(),
    )
)

config_builder.add_column(
    SamplerColumnConfig(
        name="doctor_sampler",
        sampler_type=SamplerType.PERSON,
        params=PersonSamplerParams(),
    )
)

## 🏗️ Defining Data Structure

Now we'll define the structure of our dataset by adding columns for patient information, dates, and medical details. We'll use:

- `uuid` for patient identification
- Patient personal information (`first_name`, `last_name`, `dob`, `patient_email`)
- Medical timeline information (`symptom_onset_date`, `date_of_visit`)
- Physician information (`physician`)

In [ ]:
config_builder.add_column(
    SamplerColumnConfig(
        name="patient_id",
        sampler_type=SamplerType.UUID,
        params=UUIDSamplerParams(prefix="PT-", short_form=True, uppercase=True),
    )
)

config_builder.add_column(
    name="first_name",
    column_type="expression",
    expr="{{patient_sampler.first_name}}"
)

config_builder.add_column(
    name="last_name",
    column_type="expression",
    expr="{{patient_sampler.last_name}}"
)

config_builder.add_column(
    name="dob",
    column_type="expression",
    expr="{{patient_sampler.birth_date}}"
)

config_builder.add_column(
    name="patient_email",
    column_type="expression",
    expr="{{patient_sampler.email_address}}"
)

config_builder.add_column(
    name="symptom_onset_date",
    column_type="sampler",
    sampler_type="datetime",
    params={"start": "2024-01-01", "end": "2024-12-31"},
)

config_builder.add_column(
    name="date_of_visit",
    column_type="sampler",
    sampler_type="timedelta",
    params={
        "dt_min": 1,
        "dt_max": 30,
        "reference_column_name": "symptom_onset_date"
    },
)

config_builder.add_column(
    name="physician",
    column_type="expression",
    expr="Dr. {{doctor_sampler.first_name}} {{doctor_sampler.last_name}}",
)

## 🦜 LLM-Generated Physician Notes

The final and most complex column uses an LLM to generate realistic physician notes. We provide:

- Context about the patient and their condition
- Patient summary from our seed data
- Clear formatting instructions

This will create detailed medical notes that reflect the patient's diagnosis and visit information. 

In [ ]:
config_builder.add_column(
    LLMTextColumnConfig(
        name="physician_notes",
        system_prompt=SYSTEM_PROMPT,
        model_alias=MODEL_ALIAS,
        prompt=(
            "<context>"
            "You are a primary-care physician who just had an appointment with {{first_name}} {{last_name}}, "
            "who has been struggling with symptoms from {{diagnosis}} since {{symptom_onset_date}}.\n"
            "The date of today's visit is {{date_of_visit}}.\n"
            "</context>\n"

            "<patient_summary_of_symptoms>\n"
            "{{patient_summary}}\n"
            "</patient_summary_of_symptoms>\n"

            "<task>\n"
            "Write careful notes about your visit with {{first_name}}, as {{physician}}.\n"

            "Format the notes as a busy doctor might.\n"
            "</task>"
        )
    )
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.

In [ ]:
# Preview a few records
preview = data_designer_client.preview(config_builder)

In [ ]:
# More previews
preview.display_sample_record()

### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [ ]:
# Print the analysis as a table.
preview.analysis.to_report()

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [ ]:
job_results = data_designer_client.create(config_builder, num_records=20)

# This will block until the job is complete.
job_results.wait_until_done()

In [ ]:
# Load the generated dataset as a pandas DataFrame.
dataset = job_results.load_dataset()

dataset.head()

In [ ]:
# Load the analysis results into memory.
analysis = job_results.load_analysis()

analysis.to_report()

In [ ]:
TUTORIAL_OUTPUT_PATH = "data-designer-tutorial-output"

# Download the job artifacts and save them to disk.
job_results.download_artifacts(
    output_path=TUTORIAL_OUTPUT_PATH,
    artifacts_folder_name="artifacts-community-contributions-healthcare-datasets-physician-notes",
);